In [1]:
from functions_database_updated import *
import nengo
import nengo_dl

In [2]:
inp = tf.keras.Input(shape=(2))
dense1 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(inp)
dense2 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(dense1)
output = layers.Dense(1, use_bias = True)(dense2)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

In [3]:
onnx_model_path = '../controller_single_pendulum.onnx'
onnx_model = onnx.load(onnx_model_path)
params_array = []

In [4]:
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    params_array.append(tensor_array)

In [5]:
weights_list=[]
bias_list=[]
i=0
for tensor_array in params_array:
    # Convert the initializer tensor to a NumPy array
#     tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==0):
        weights_list.append(tensor_array)
    i = i+1
    

i=0
for tensor_array in params_array:
    # Convert the initializer tensor to a NumPy array
#     tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==1):
        bias_list.append(tensor_array)
    i = i+1

In [6]:
weights_list.reverse()
bias_list.reverse()

In [7]:
for i in range(1, len(weights_list) + 1):
    transposed_weights = weights_list[i-1]
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

In [8]:
weights, biases = extract_model_params_tf(nn_ctlr)

In [9]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    # Declarations
    for num in range(1,inputs+1):
        declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-99999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")

    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 999999
    eps = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                

    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}>={cond[1]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MAXIMIZE)')


    return equations, declare

In [10]:
def can_go_below(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    # Declarations
    for num in range(1,inputs+1):
        declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-99999, name='S{layer}_{no}_{time}')")

    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    eps = 0.00000001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                

    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}<={cond[0]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MINIMIZE)')

    return equations, declare

In [11]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings_5L_ACC_sim_random.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
#     import time
#     start = time.time()
    model=gp.Model("Encodings")

#     model.Params.MIPGap = GRB.INFINITY
    model.Params.MIPFocus = focus####################################################################
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1
#     model.Params.Heuristics=0
#     model.Params.PreCrush=0
#     model.tune()
#     model.Params.Threads = 1

    try:
        f = open(file_path,"r")
        try:
            for l in f:
#                 print(l)
                exec(l)
        finally:
            f.close()
    except IOError:
        pass
#     print('Time taken:', end-start)
#     model.tune()

    model.optimize()
    
    return model

In [12]:
layer_no = 3
neuron_no = 1
time_steps = 5
input_bounds = [[1.0,1.2],[0.0,0.2]]

In [13]:
output_range = [-0.781295,-0.542820]

In [14]:
uppers = [0,0,0,0,-0.294831, -0.360207 , -0.425026, -0.484878,-0.424881,-0.446818, -0.500576, -0.490154, -0.464801, -0.469808, -0.459036, -0.478375, -0.473631, -0.490891,-0.513279, -0.528757, -0.510519]

for tym in range(4,5): 
    runs_for = 0
    print('This is timestep number',tym)
    output_range = [0,-0.542820 + 0.4]

    flag = 0
    lb = -0.542820
    ub = -0.542820  + 0.4
    checked = []
    result = []
    while (flag <= 1):
    
        print("LB:", lb)
        print("UB:", ub)
        mid = (lb+ub)/2
#         mid = random.uniform(lb, ub)
        if(len(checked) > 1 and abs(ub - lb) < 0.001) :
            flag = flag + 1


        output_range[1] = mid
    #     print(output_range)
        print(tym, "Checking with UB:", output_range[1])
        
        
        if(uppers[tym] > mid):
            print('violated by 500 sims')
            lb = mid
            continue
        
        
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        checked.append(output_range[1])
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print(tym, 'Property does not hold')
            lb = mid 
        elif(model3.status in [9]):
            print(tym, 'Time out')
            break
        else:
            print(model3.status, 'Property holds')
            ub = mid
        result.append(model3.status)
        print('Runtime',model3.Runtime) 
        runs_for = runs_for + model3.Runtime
        print(checked)
        print(result)
        print("\n")
    print(runs_for)

This is timestep number 4
LB: -0.54282
UB: -0.14281999999999995
4 Checking with UB: -0.34281999999999996
violated by 500 sims
LB: -0.34281999999999996
UB: -0.14281999999999995
4 Checking with UB: -0.24281999999999995
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20

Time steps 4
4 Property holds
Runtime 0.06499981880187988
[-0.24281999999999995]
[4]


LB: -0.34281999999999996
UB: -0.24281999999999995
4 Checking with UB: -0.29281999999999997

Time steps 4
4 Property holds
Runtime 0.04200005531311035
[-0.24281999999999995, -0.29281999999999997]
[4, 4]


LB: -0.34281999999999996
UB: -0.29281999999999997
4 Checking with UB: -0.31782
violated by 500 sims
LB: -0.31782
UB: -0.29281999999999997
4 Checking with UB: -0.30532
violated by 500 sims
LB: -0.30532
UB: -0.29281999999999997
4 Checking with UB: -0.29906999999999995
violated by 500 sims
LB: -0.29906999999999995
UB: -0.29281999999999997
4 Checking with UB: -0.29594499999999996
violated by 500 sims


In [ ]:
uppers = [0,0,0,0,-0.294831, -0.360207 , -0.425026, -0.484878,-0.424881,-0.446818, -0.500576, -0.490154, -0.464801, -0.469808, -0.459036, -0.478375, -0.473631, -0.490891,-0.513279, -0.528757, -0.510519]

for tym in range(5,21): 
    runs_for = 0
    print('This is timestep number',tym)
    output_range = [0,-0.542820 + 0.2]

    flag = 0
    lb = -0.542820
    ub = -0.542820  + 0.2
    checked = []
    result = []
    while (flag <= 1):
    
        print("LB:", lb)
        print("UB:", ub)
        mid = (lb+ub)/2
#         mid = random.uniform(lb, ub)
        if(len(checked) > 1 and abs(ub - lb) < 0.001) :
            flag = flag + 1


        output_range[1] = mid
    #     print(output_range)
        print(tym, "Checking with UB:", output_range[1])
        
        
        if(uppers[tym] > mid):
            print('violated by 500 sims')
            lb = mid
            continue
        
        
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        checked.append(output_range[1])
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print(tym, 'Property does not hold')
            lb = mid 
        elif(model3.status in [9]):
            print(tym, 'Time out')
            break
        else:
            print(model3.status, 'Property holds')
            ub = mid
        result.append(model3.status)
        print('Runtime',model3.Runtime) 
        runs_for = runs_for + model3.Runtime
        print(checked)
        print(result)
        print("\n")
    print(runs_for)

This is timestep number 5
LB: -0.54282
UB: -0.34281999999999996
5 Checking with UB: -0.44282
violated by 500 sims
LB: -0.44282
UB: -0.34281999999999996
5 Checking with UB: -0.39281999999999995
violated by 500 sims
LB: -0.39281999999999995
UB: -0.34281999999999996
5 Checking with UB: -0.3678199999999999
violated by 500 sims
LB: -0.3678199999999999
UB: -0.34281999999999996
5 Checking with UB: -0.35531999999999997

Time steps 5
4 Property holds
Runtime 0.04699993133544922
[-0.35531999999999997]
[4]


LB: -0.3678199999999999
UB: -0.35531999999999997
5 Checking with UB: -0.36156999999999995
violated by 500 sims
LB: -0.36156999999999995
UB: -0.35531999999999997
5 Checking with UB: -0.35844499999999996

Time steps 5
4 Property holds
Runtime 0.06700015068054199
[-0.35531999999999997, -0.35844499999999996]
[4, 4]


LB: -0.36156999999999995
UB: -0.35844499999999996
5 Checking with UB: -0.3600074999999999

Time steps 5
4 Property holds
Runtime 0.07800006866455078
[-0.35531999999999997, -0.3584449


Time steps 10
4 Property holds
Runtime 18.73300004005432
[-0.44282, -0.49282, -0.49907, -0.49985125]
[4, 4, 4, 4]


LB: -0.5006325
UB: -0.49985125
10 Checking with UB: -0.5002418750000001

Time steps 10
4 Property holds
Runtime 16.052000045776367
[-0.44282, -0.49282, -0.49907, -0.49985125, -0.5002418750000001]
[4, 4, 4, 4, 4]


LB: -0.5006325
UB: -0.5002418750000001
10 Checking with UB: -0.5004371875

Time steps 10
4 Property holds
Runtime 18.95199990272522
[-0.44282, -0.49282, -0.49907, -0.49985125, -0.5002418750000001, -0.5004371875]
[4, 4, 4, 4, 4, 4]


100.11400032043457
This is timestep number 11
LB: -0.54282
UB: -0.34281999999999996
11 Checking with UB: -0.44282

Time steps 11
4 Property holds
Runtime 21.592000007629395
[-0.44282]
[4]


LB: -0.54282
UB: -0.44282
11 Checking with UB: -0.49282
violated by 500 sims
LB: -0.49282
UB: -0.44282
11 Checking with UB: -0.46782

Time steps 11
4 Property holds
Runtime 13.703999996185303
[-0.44282, -0.46782]
[4, 4]


LB: -0.49282
UB: -0.4678


Time steps 15
15 Property does not hold
Runtime 6.558000087738037
[-0.44282, -0.39281999999999995, -0.41781999999999997, -0.43032, -0.43657, -0.439695]
[10, 4, 4, 4, 4, 10]


LB: -0.439695
UB: -0.43657
15 Checking with UB: -0.43813250000000004

Time steps 15
15 Property does not hold
Runtime 18.55999994277954
[-0.44282, -0.39281999999999995, -0.41781999999999997, -0.43032, -0.43657, -0.439695, -0.43813250000000004]
[10, 4, 4, 4, 4, 10, 10]


LB: -0.43813250000000004
UB: -0.43657
15 Checking with UB: -0.43735125

Time steps 15
15 Property does not hold
Runtime 20.70899987220764
[-0.44282, -0.39281999999999995, -0.41781999999999997, -0.43032, -0.43657, -0.439695, -0.43813250000000004, -0.43735125]
[10, 4, 4, 4, 4, 10, 10, 10]


LB: -0.43735125
UB: -0.43657
15 Checking with UB: -0.436960625

Time steps 15
4 Property holds
Runtime 84.47099995613098
[-0.44282, -0.39281999999999995, -0.41781999999999997, -0.43032, -0.43657, -0.439695, -0.43813250000000004, -0.43735125, -0.436960625]
[10, 4,


Time steps 19
4 Property holds
Runtime 2973.2240002155304
[-0.44282, -0.49282, -0.51782, -0.5240699999999999, -0.5271949999999999, -0.5279762499999998, -0.5283668749999999, -0.5285621875]
[4, 4, 4, 4, 4, 4, 4, 4]


9453.935000419617
This is timestep number 20
LB: -0.54282
UB: -0.34281999999999996
20 Checking with UB: -0.44282

Time steps 20
4 Property holds
Runtime 1395.5620000362396
[-0.44282]
[4]


LB: -0.54282
UB: -0.44282
20 Checking with UB: -0.49282

Time steps 20
4 Property holds
Runtime 1580.888000011444
[-0.44282, -0.49282]
[4, 4]


LB: -0.54282
UB: -0.49282
20 Checking with UB: -0.51782
violated by 500 sims
LB: -0.51782
UB: -0.49282
20 Checking with UB: -0.50532

Time steps 20
4 Property holds
Runtime 1191.9470000267029
[-0.44282, -0.49282, -0.50532]
[4, 4, 4]


LB: -0.51782
UB: -0.50532
20 Checking with UB: -0.51157
violated by 500 sims
LB: -0.51157
UB: -0.50532
20 Checking with UB: -0.508445

Time steps 20
4 Property holds
Runtime 3979.315999984741
[-0.44282, -0.49282, -0.

In [14]:
for tym in range(5,21):   
    print('This is timestep number',tym)
    output_range = [0,-0.542820 + 0.2]

    flag = 0
    lb = -0.542820
    ub = -0.542820  + 0.2
    checked = []
    result = []
    while (flag <= 1):
    #     mid = (lb+ub)/2
        print("LB:", lb)
        print("UB:", ub)
        mid = random.uniform(lb, ub)
        if(len(checked) > 1 and abs(ub - lb) < 0.001) :
            flag = flag + 1


        output_range[1] = mid
    #     print(output_range)
        print(tym, "Checking with UB:", output_range[1])
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,1,6,0)
        checked.append(output_range[1])
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print(tym, 'Property does not hold')
            lb = mid 
        elif(model3.status in [9]):
            print(tym, 'Time out')
            break
        else:
            print(model3.status, 'Property holds')
            ub = mid
        result.append(model3.status)
        print('Runtime',model3.Runtime)    
        print(checked)
        print(result)
        print("\n")

This is timestep number 5
LB: -0.54282
UB: -0.34281999999999996
5 Checking with UB: -0.4069817488468961
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2057 rows, 1308 columns and 8311 nonzeros
Model fingerprint: 0x40864d64
Variable types: 808 continuous, 500 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 2003 rows and 1274 columns
Presolve time: 0.15s
Presolved: 54 rows, 34 columns, 181 nonzeros
Variable types: 21 continuous, 13 integer (10 binary)
Found heuristic solution

Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 2032 rows and 1292 columns
Presolve time: 0.08s

Explored 0 nodes (0 simplex iterations) in 0.10 seconds (0.03 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 5
4 Property holds
Runtime 0.1119999885559082
[-0.4069817488468961, -0.35691261840181154, -0.36589602538738303, -0.3602473093089537, -0.35787381141978214, -0.3598534693361017, -0.36012581812169175]
[2, 4, 2, 2, 4, 4, 4]


LB: -0.3602473093089537
UB: -0.36012581812169175
5 Checking with UB: -0.3601392420485495
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread coun

Presolved: 148 rows, 104 columns, 651 nonzeros
Variable types: 68 continuous, 36 integer (28 binary)
Found heuristic solution: objective -2.5501563

Explored 0 nodes (0 simplex iterations) in 0.11 seconds (0.04 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -2.55016 

Solution limit reached
Best objective -2.550156299956e+00, best bound -1.061713580973e+00, gap 58.3667%

Time steps 6
6 Property does not hold
Runtime 0.11300015449523926
[-0.5140057230369519, -0.3553167355102585, -0.4307575218284363, -0.4166111384173568, -0.4269663917173291]
[10, 4, 10, 4, 10]


LB: -0.4269663917173291
UB: -0.4166111384173568
6 Checking with UB: -0.4185222289658253
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize

Variable types: 959 continuous, 600 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 2310 rows and 1455 columns
Presolve time: 0.07s
Presolved: 148 rows, 104 columns, 651 nonzeros
Variable types: 68 continuous, 36 integer (28 binary)

Root relaxation: objective -1.839857e+00, 75 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -1.83986    0   15          -   -1.83986      -     -    0s
     0     0 -1.000e+100    0   16          - infeasible      -     -    0s

Cutting planes:
  Learned: 1
  Cover: 9
  Implied bound: 16
  Clique: 2
  MIR: 18
  StrongCG: 1
  GUB cover: 1
  RLT: 2
  Relax-and-lift: 1

Explored 1 nodes (107 simplex iterations) in 0.13 seconds (0.04 work units)
Thread 

Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2859 rows, 1810 columns and 11615 nonzeros
Model fingerprint: 0xa2b6832e
Variable types: 1110 continuous, 700 integer (350 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 2613 rows and 1627 columns
Presolve time: 0.08s
Presolved: 246 rows, 183 columns, 1094 nonzeros
Variable types: 116 continuous, 67 integer (55 binary)
Found heuristic solution: objective -3.3941470

Explored 0 nodes (0 simplex iterations) in 0.11 seconds (0.05 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -3.39415 

Solution limit reached
Best objective -3.394147020299e+00, best bound -2.347166766413e-01, gap 93.0847%

Time steps 7
7 Property does not hold
Runtime 0.12200021743774414
[-0.5013167299519453, -0.4038789555278778, -0.4630422230833312, -0.48996

...

 248002 16459 infeasible   47               -   -9.67957      -  66.6 2951s
 248391 16188 infeasible   44               -   -9.68455      -  66.6 2958s
 248776 16106   -9.80033   59  104          -   -9.69127      -  66.7 2972s
 248902 15860  -10.18123   41  127          -   -9.69243      -  66.7 2977s
 249242 15541 infeasible   49               -   -9.69575      -  66.7 2984s
 249655 15200   -9.94502   50  127          -   -9.70569      -  66.8 2991s
 250090 14866 infeasible   52               -   -9.71073      -  66.8 2998s
 250520 14479 infeasible   64               -   -9.72003      -  66.9 3004s
 250989 14190  -10.11235   52   75          -   -9.72832      -  66.9 3012s
 251463 13802   -9.91213   61   64          -   -9.73125      -  67.0 3020s
 251980 13452 infeasible   41               -   -9.74510      -  67.0 3028s
 252604 13881 infeasible   46               -   -9.75275      -  67.0 3037s
 256196 13470 infeasible   45               -   -9.76541      -  66.4 3046s
 256727 1293

 10885  4824 infeasible   32               -   -7.61599      -  66.7  116s
 11700  5072   -8.95277   31  145          -   -7.65308      -  65.7  121s
 12717  5376   -7.82494   25  156          -   -7.71390      -  64.9  125s
 13357  5688   -8.65066   33  151          -   -7.73568      -  64.8  130s
 14271  5959   -8.45725   28  154          -   -7.77648      -  64.5  135s
 15127  6406   -9.96086   31  178          -   -7.81602      -  63.7  140s
 16651  6403 infeasible   23               -   -7.86487      -  62.8  152s
 16752  6401 infeasible   27               -   -7.87715      -  63.5  157s
 17822  6741 infeasible   45               -   -7.94068      -  63.0  162s
 18757  7066 infeasible   35               -   -7.97260      -  63.1  167s
 18857  7357   -8.72354   32  153          -   -7.98237      -  63.5  170s
 19915  7651   -8.29080   30  150          -   -8.00068      -  63.3  176s
 21066  7974   -8.37985   28  144          -   -8.03768      -  63.2  182s
 22188  7969   -9.15414  

 43930 12514 infeasible   36               -   -8.49603      -  63.7  716s
 45219 12870 infeasible   58               -   -8.52288      -  63.0  725s
 45853 13491   -8.78392   44  169          -   -8.54756      -  63.0  730s
 48013 12798   -9.47076   52  108          -   -8.58437      -  62.5  741s
 48181 12728   -8.79081   52  128          -   -8.58832      -  62.6  748s
 48262 13175  -10.16402   47  119          -   -8.58854      -  62.9  751s
 49599 12731   -8.73728   50  151          -   -8.60081      -  62.5  758s
 49699 13065   -8.99922   51  145          -   -8.60148      -  62.6  761s
 50985 12624   -8.97779   45  144          -   -8.61527      -  62.2  765s
 52245 12625   -8.80964   57  151          -   -8.62638      -  62.3  771s
 53350 12577 infeasible   50               -   -8.63670      -  62.3  777s
 53450 12841 infeasible   56               -   -8.63951      -  62.6  780s
 54419 12805   -8.86073   51  117          -   -8.64632      -  62.9  787s
 55293 12521   -9.51460  

 127343 19799 infeasible   59               -   -9.05686      -  76.3 1521s
 127491 19745 infeasible   62               -   -9.05950      -  76.4 1525s
 127803 19610   -9.39322   48  168          -   -9.06298      -  76.7 1533s
 127892 19543   -9.77557   49  145          -   -9.06298      -  76.9 1537s
 128043 19483 infeasible   50               -   -9.06476      -  77.0 1546s
 128209 19424 infeasible   68               -   -9.06750      -  77.2 1553s
 128364 19313   -9.58459   52  121          -   -9.06951      -  77.3 1561s
 128551 19247 infeasible   58               -   -9.06951      -  77.5 1565s
 128893 19131 infeasible   61               -   -9.07548      -  77.6 1570s
 129198 19305 infeasible   65               -   -9.08088      -  77.7 1576s
 130763 19180 infeasible   75               -   -9.08604      -  77.4 1581s
 130998 19067 infeasible   50               -   -9.08878      -  77.5 1586s
 131169 19000 infeasible   66               -   -9.09198      -  77.7 1596s
 131270 1887

 189855  4274 infeasible   67               -   -9.96656      -  85.6 2521s
 190452  3757 infeasible   47               -   -9.98015      -  85.7 2532s
 191105  3312  -10.03664   44  165          -  -10.02000      -  85.8 2542s
 191794  2803 infeasible   55               -  -10.05478      -  85.9 2552s
 192513  2200 infeasible   47               -  -10.07912      -  86.0 2564s
 193336  1929 infeasible   65               -  -10.10450      -  86.1 2585s
 193689  1008 infeasible   89               -  -10.10862      -  86.2 2596s
 194665     0 infeasible   47               -  -10.20174      -  86.2 2601s

Cutting planes:
  Learned: 36
  Gomory: 23
  Cover: 179
  Implied bound: 103
  Projected implied bound: 20
  Clique: 11
  MIR: 921
  StrongCG: 66
  Flow cover: 787
  Inf proof: 29
  Zero half: 34
  RLT: 14
  Relax-and-lift: 58

Explored 195673 nodes (16816081 simplex iterations) in 2602.32 seconds (653.16 work units)
Thread count was 8 (of 8 available processors)

Solution count 0
No othe

In [ ]:
0.24300003051757812 + 0.24300003051757812+ 0.2590000629425049+

In [16]:
for tym in range(1,6):   
    print('This is timestep number',tym)
    output_range = [0,-0.542820 + 0.9]

    flag = 0
    lb = -0.542820
    ub = -0.542820  + 0.9
    checked = []
    result = []
    while (flag <= 1):
    #     mid = (lb+ub)/2
        print("LB:", lb)
        print("UB:", ub)
        mid = random.uniform(lb, ub)
        if(len(checked) > 1 and abs(ub - lb) < 0.001) :
            flag = flag + 1


        output_range[1] = mid
    #     print(output_range)
        print(tym, "Checking with UB:", output_range[1])
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,1,6,0)
        checked.append(output_range[1])
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print(tym, 'Property does not hold')
            lb = mid
        elif(model3.status in [9]):
            print(tym, 'Time out')
            break
        else:
            print(model3.status, 'Property holds')
            ub = mid
        result.append(model3.status)
        print('Runtime',model3.Runtime)    
        print(checked)
        print(result)
        print("\n")

This is timestep number 1
LB: -0.54282
UB: 0.35718000000000005
1 Checking with UB: -0.5085118519073998
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 453 rows, 304 columns and 1703 nonzeros
Model fingerprint: 0xd437eac6
Variable types: 204 continuous, 100 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 453 rows and 304 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0.00490048 
No other solutions better t


Time steps 1
4 Property holds
Runtime 0.04700016975402832
[-0.5085118519073998, -0.5021141101174251, 0.1633442374288948, 0.12128360807115712, -0.011756950625939233, 0.0027011824871762662, -0.0016846226739382283]
[2, 2, 4, 4, 2, 4, 4]


LB: -0.011756950625939233
UB: -0.0016846226739382283
1 Checking with UB: -0.005912727258501556
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 453 rows, 304 columns and 1703 nonzeros
Model fingerprint: 0x449bbaca
Variable types: 204 continuous, 100 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 453 rows and 304 columns
Presolve tim

Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 854 rows, 555 columns and 3355 nonzeros
Model fingerprint: 0xdcbf93e2
Variable types: 355 continuous, 200 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 854 rows and 555 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0.00980097 
No other solutions better than -0.00980097

Optimal solution found (tolerance 1.00e-04)
Best objective -9.800969623029e-03, best bound -9.800969623029e-03, gap 0.0000%



Solution count 1: -0.00980097 
No other solutions better than -0.00980097

Optimal solution found (tolerance 1.00e-04)
Best objective -9.800969623029e-03, best bound -9.800969623029e-03, gap 0.0000%

Time steps 2
2 Property does not hold
Runtime 0.06399989128112793
[-0.1808532381199412, -0.08375211246861296, 0.2615768759759177, 0.003823039950740084, -0.014234043069751193, -0.013869129885714529, -0.011922707051332969]
[2, 2, 4, 4, 2, 2, 2]


LB: -0.011922707051332969
UB: 0.003823039950740084
2 Checking with UB: -0.003995819921695206
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 854 rows, 555 columns and 3355 nonzeros
Model fingerprint: 0x52e7b53e
Variable types: 355 continuous, 200 integer (100 binary)
Coeffi


Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0.00980097 
No other solutions better than -0.00980097

Optimal solution found (tolerance 1.00e-04)
Best objective -9.800969623029e-03, best bound -9.800969623029e-03, gap 0.0000%

Time steps 2
2 Property does not hold
Runtime 0.059999942779541016
[-0.1808532381199412, -0.08375211246861296, 0.2615768759759177, 0.003823039950740084, -0.014234043069751193, -0.013869129885714529, -0.011922707051332969, -0.003995819921695206, -0.009222655097957484, -0.005365709213946079, -0.004774606906478655, -0.005163751561315458, -0.005149326553187336]
[2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 2, 2]


This is timestep number 3
LB: -0.54282
UB: 0.35718000000000005
3 Checking with UB: -0.09316613941072799
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-106

...

  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve removed 1581 rows and 1007 columns
Presolve time: 0.04s

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 4
4 Property holds
Runtime 0.08299994468688965
[-0.39081907012934447, -0.25161022499924357, -0.3165313434548204, -0.29085092728671963, -0.3109887931897115, -0.3095910948669444, -0.29390800690086233, -0.30097383700901814, -0.3002050458174662, -0.2997188190301072, -0.29841209170900596, -0.29407745283697373]
[2, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4]


LB: -0.29841209170900596
UB: -0.29407745283697373
4 Checking with UB: -0.29559774357271
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SS


Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 4
4 Property holds
Runtime 0.08100008964538574
[-0.39081907012934447, -0.25161022499924357, -0.3165313434548204, -0.29085092728671963, -0.3109887931897115, -0.3095910948669444, -0.29390800690086233, -0.30097383700901814, -0.3002050458174662, -0.2997188190301072, -0.29841209170900596, -0.29407745283697373, -0.29559774357271, -0.29556341652297413, -0.29550092491817614, -0.2950963523806172, -0.29408757243141515]
[2, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4]


LB: -0.2950963523806172
UB: -0.29408757243141515
4 Checking with UB: -0.2946948102887197
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1656 rows, 105

Presolve removed 1972 rows and 1256 columns
Presolve time: 0.07s
Presolved: 85 rows, 52 columns, 333 nonzeros
Variable types: 32 continuous, 20 integer (15 binary)
Found heuristic solution: objective -1.8010345

Explored 0 nodes (0 simplex iterations) in 0.10 seconds (0.03 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -1.80103 

Solution limit reached
Best objective -1.801034455653e+00, best bound -1.056813096162e+00, gap 41.3219%

Time steps 5
5 Property does not hold
Runtime 0.11599993705749512
[0.2879312584621, 0.15422406239597264, -0.4457229451671622]
[4, 4, 10]


LB: -0.4457229451671622
UB: 0.15422406239597264
5 Checking with UB: -0.143858947079671
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads




Time steps 5
5 Property does not hold
Runtime 0.1269998550415039
[0.2879312584621, 0.15422406239597264, -0.4457229451671622, -0.143858947079671, -0.25560711903166006, -0.33166043513079574, -0.43520377722254, -0.4234125989492997, -0.3707060876642562]
[4, 4, 10, 4, 4, 4, 10, 2, 2]


LB: -0.3707060876642562
UB: -0.33166043513079574
5 Checking with UB: -0.34794226430729036
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2057 rows, 1308 columns and 8311 nonzeros
Model fingerprint: 0xf37ab2b1
Variable types: 808 continuous, 500 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 1e+05]
  RHS range        [6e-04, 1e+06]
Presolve r

Presolve time: 0.06s
Presolved: 8 rows, 6 columns, 22 nonzeros
Variable types: 2 continuous, 4 integer (2 binary)
Found heuristic solution: objective -1.8010345

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.03 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -1.80103 
No other solutions better than -1.80103

Optimal solution found (tolerance 1.00e-04)
Best objective -1.801034455653e+00, best bound -1.801034455653e+00, gap 0.0000%

Time steps 5
5 Property does not hold
Runtime 0.11599993705749512
[0.2879312584621, 0.15422406239597264, -0.4457229451671622, -0.143858947079671, -0.25560711903166006, -0.33166043513079574, -0.43520377722254, -0.4234125989492997, -0.3707060876642562, -0.34794226430729036, -0.35045440746861506, -0.35960819996640775, -0.3703614793884621, -0.36313956189865626, -0.36161396641932564]
[4, 4, 10, 4, 4, 4, 10, 2, 2, 4, 4, 4, 2, 2, 2]


LB: -0.36161396641932564
UB: -0.35960819996640775
5 Checking with UB: -0.3605133616153381
